In [1]:
## Loading packages
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PyDictionary import PyDictionary # Dictionary to get meaning
from sklearn.feature_extraction.text import TfidfVectorizer # word to vector
from nltk.stem.snowball import SnowballStemmer # stemming
from scipy.sparse import hstack 
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
import re

# 

from sklearn.feature_extraction.text import TfidfVectorizer


dictionary=PyDictionary() # dictionary file to get dictionary

In [2]:
# loading data
data = pd.read_csv('classkeywords.csv')

In [3]:
data.head(10)

,ClassDescKeywordId,Keyword,ClassDescriptionId,ClassCode,ClassSuffix,DirectOK
0,1,Automobile Parts and Accessories,1,8046,1,Y
1,316,Auto Parts,1,8046,1,Y
2,317,Used Automotive Parts,1,8046,1,Y
3,327,Aftermarket Auto Parts,1,8046,1,Y
4,328,Car Batteries,1,8046,1,Y
5,345,Car Parts and Accessories,1,8046,1,Y
6,344,Truck Stops,2,8391,2,Y
7,382,Petrol Station,2,8391,2,Y
8,383,Filling Station,2,8391,2,Y
9,384,Service Station,2,8391,2,Y


In [4]:
## Using dictionary meaning from pydictionary to make several representations for the same keyword
## only using nouns. 
print dictionary.meaning("flower")

{u'Verb': ['produce or yield flowers'], u'Noun': ['a plant cultivated for its blooms or blossoms', 'reproductive organ of angiosperm plants especially one having showy or colorful parts', 'the period of greatest prosperity or productivity']}


//anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [5]:
## stop words are common words in english. 
## These words are expected to not contain much information about data. 
sw = stopwords.words('english')

In [6]:
# converting adding meaning to keywords
# if the word is not in dictionary, it returns error. 
#for i in range(0,len(data)):
meaning_list = []
KW_list = []

for i in range(0,len(data)):
    if (i % 100)==0:
        print i
    meaning_str = ''
    kw_wo_spl = re.sub('[^A-Za-z0-9]+', ' ', data['Keyword'][i]).lower()
    kw = kw_wo_spl.split(' ')
    for word in kw:
        if ((word in sw) == False):
            try:
                #print word + ', OK'
                meaning_wd = dictionary.meaning(word)
                if 'Noun' in meaning_wd.keys():
                    meaning_str = meaning_str + meaning_wd['Noun'][0] + " "
                if 'Verb' in meaning_wd.keys():
                    meaning_str = meaning_str + meaning_wd['Verb'][0] + " "
            except:
                print word
                pass
    KW_sub = kw
    meaning_str = meaning_str[0:len(meaning_str)-1]
    #print meaning_str
    meaning_list.append(meaning_str)
    KW_list.append(kw_wo_spl)


0
Error: The Following Error occured: list index out of range
aftermarket
Error: The Following Error occured: list index out of range
customization
Error: The Following Error occured: list index out of range
customization
Error: The Following Error occured: list index out of range
customization
Error: The Following Error occured: list index out of range
handyperson
Error: The Following Error occured: list index out of range
remodeler
Error: The Following Error occured: list index out of range
sealcoating
Error: The Following Error occured: list index out of range
hvac
Error: The Following Error occured: list index out of range
hvac
Error: The Following Error occured: list index out of range
drywalling
Error: The Following Error occured: list index out of range
lathing
100
Error: The Following Error occured: list index out of range
ymca
Error: The Following Error occured: list index out of range
ywca
Error: The Following Error occured: list index out of range
ymha
Error: The Following E

In [8]:
# vectorize meaning of keywords using tf-idf
vectorizer_meaning = TfidfVectorizer(use_idf=True,
                                         stop_words='english').fit(meaning_list)
tf_meaning = vectorizer_meaning.transform(meaning_list)

In [9]:
# vectorize words using tf-idf
vectorizer_keyword = TfidfVectorizer(use_idf=True,
                                         stop_words='english').fit(KW_list)
tf_keyword = vectorizer_keyword.transform(KW_list)


In [10]:
tf_labels = np.array(data['ClassCode'])

In [11]:
tf_features = hstack([tf_meaning,tf_keyword]).toarray()

In [51]:
# simple logistic regression model
clf = LogisticRegression()
clf.fit(tf_features,tf_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [52]:
precision = precision_score(clf.predict(tf_features),tf_labels)
recall = recall_score(clf.predict(tf_features),tf_labels)
print 'accuracy = ' + str(float(sum(clf.predict(tf_features)==tf_labels))/float(len(tf_labels))*100) + '%'
print 'precision = ' + str(precision*100) + '%'
print 'recall = ' + str(recall*100) + '%'

accuracy = 91.40625%
precision = 97.1668061756%
recall = 91.40625%


//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1203: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sampl

In [53]:
confusion_matrix(clf.predict(tf_features),tf_labels)

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 3, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 7, 0, 0],
       [0, 0, 0, ..., 0, 2, 0],
       [0, 0, 0, ..., 0, 0, 4]])

In [54]:
## Predicting probabilities
prob_pred = clf.predict_proba(tf_features)
prob_pred

array([[ 0.00449993,  0.00564016,  0.00474343, ...,  0.00799134,
         0.00507781,  0.00617686],
       [ 0.00418352,  0.00518072,  0.00431578, ...,  0.00718065,
         0.00468743,  0.00564287],
       [ 0.00488051,  0.00629069,  0.0050141 , ...,  0.00881786,
         0.00552469,  0.00676024],
       ..., 
       [ 0.00491105,  0.00630185,  0.00506821, ...,  0.00933139,
         0.0056013 ,  0.00698385],
       [ 0.00484929,  0.00615565,  0.00497033, ...,  0.01113732,
         0.00580014,  0.00678957],
       [ 0.00508664,  0.0064391 ,  0.00507602, ...,  0.01006713,
         0.00597223,  0.00709049]])

In [55]:
data['predictions'] = clf.predict(tf_features)
data[['predictions','ClassCode']]

,predictions,ClassCode
0,8046,8046
1,8046,8046
2,8046,8046
3,8046,8046
4,8391,8046
5,8046,8046
6,8391,8391
7,8391,8391
8,8391,8391
9,8391,8391


In [58]:
## Enter NEW KEYWORD here.

kw_search = 'truck parts'
meaning_list=[]
KW_list=[]
for i in range(0,1):
    meaning_str = ''
    kw_wo_spl = re.sub('[^A-Za-z0-9]+', ' ', kw_search).lower()
    kw = kw_wo_spl.split(' ')
    for word in kw:
        if ((word in sw) == False):
            try:
                #print word + ', OK'
                meaning_wd = dictionary.meaning(word)
                if 'Noun' in meaning_wd.keys():
                    meaning_str = meaning_str + meaning_wd['Noun'][0] + " "
                if 'Verb' in meaning_wd.keys():
                    meaning_str = meaning_str + meaning_wd['Verb'][0] + " "
            except:
                print word
                pass
    KW_sub = kw
    meaning_str = meaning_str[0:len(meaning_str)-1]
    #print meaning_str
    meaning_list.append(meaning_str)
    KW_list.append(kw_wo_spl)
tf_meaning_wd = vectorizer_meaning.transform(meaning_list)
tf_keyword_wd = vectorizer_keyword.transform(KW_list)
tf_features_wd = hstack([tf_meaning_wd,tf_keyword_wd]).toarray()
print clf.predict(tf_features_wd)
print 'predicted class for "' + kw_search + '" is ' + str(clf.predict(tf_features_wd)[0])

[8046]
predicted class for "truck parts" is 8046


In [59]:
print clf.predict_proba(tf_features_wd)

[[ 0.00489801  0.00616049  0.00505773  0.00573838  0.00660532  0.00482171
   0.00647766  0.00487709  0.00478041  0.0047461   0.00549925  0.00484999
   0.00499489  0.0047085   0.00475416  0.00542734  0.01216766  0.00477949
   0.00479202  0.00483361  0.00478113  0.00557427  0.00554664  0.00479757
   0.00477429  0.0047669   0.00658082  0.00495722  0.00644962  0.00482887
   0.00717536  0.00796429  0.00574487  0.00662457  0.00486233  0.00548638
   0.01048331  0.00734025  0.00494223  0.00555122  0.00541398  0.00618673
   0.0054163   0.00551751  0.00647435  0.00482685  0.00674921  0.0066851
   0.00618354  0.00489984  0.00476093  0.00543674  0.00759096  0.00666995
   0.00553571  0.00685423  0.00491095  0.00488591  0.00609502  0.00770082
   0.00491032  0.00768477  0.00841206  0.00488725  0.00491181  0.02030669
   0.00594113  0.00502412  0.00489772  0.00541096  0.0790548   0.00536656
   0.00489307  0.00621642  0.00486615  0.00477307  0.0048562   0.00494732
   0.00552761  0.00554106  0.00693359  